## This Jupyter notebook lets you donwload 3D data (Lat x Lon x Time) dataset from the Ocean Colour - Climate Change Initiave (OC-CCI) using OPENDAP

## Works for the OC-CCI v5 daily dataset (but you can adapt to previous versions)


In [1]:
import os #change folders
import numpy as np # perform calculations and basic math
import matplotlib.pyplot as plt # plot data
import pandas as pd # work with dataframes,tables, spreadsheets, etc.
import netCDF4 as nc4 # work with netcdf files, the standard file for satellite 2D and 3D data
import cartopy #work with geographical projections and maps
import datetime # this library is useful for working with dates, convert dates, etc.
from pathlib import Path # Work with paths/directories

## Define all the functions required (you can ignore this part)

In [2]:
def serial_date_to_string(srl_no):
    """Converts CCI serial number time to datetime"""
    new_date = datetime.datetime(1970, 1, 1, 0, 0) + datetime.timedelta(srl_no)
    return new_date
def define_ROI(lat_upper, lat_lower, lon_upper, lon_lower):
    """Defines Region of Interest latitude and longitude to download"""
    try:
        lat_upper = int(lat_upper)
        lat_lower = int(lat_lower)
        lon_upper = int(lon_upper)
        lon_lower = int(lon_lower)
    except ValueError:
        print("ERROR: Please enter correct latitude/longitude")
    LATBD = [lat_upper, lat_lower]
    LONBD = [lon_lower, lon_upper]
    print('Your region of interest is:')
    print('Latitude:', LATBD)
    print('Longitude:', LONBD)
    return LATBD, LONBD
def define_time(time_init, time_final):
    """Defines period of time of interest to download"""
    try:
        time_start_year = int(time_init[0:4])
        time_start_month = int(time_init[5:7])
        time_start_day = int(time_init[8:10])
    except ValueError:
        print("ERROR: Please enter correct initial date")
    try:
        time_end_year = int(time_final[0:4])
        time_end_month = int(time_final[5:7])
        time_end_day = int(time_final[8:10])
    except ValueError:
        print("ERROR: Please enter correct final date")
    time_start_datetime = datetime.datetime(time_start_year, time_start_month, time_start_day)
    time_end_datetime = datetime.datetime(time_end_year, time_end_month, time_end_day)
    time_diff = (time_end_datetime-time_start_datetime).days+1
    print('Your data timespan is:')
    print('Initial date:', time_start)
    print('Final date:', time_end)
    print('That corresponds to', time_diff, 'days.')
    return time_start_datetime, time_end_datetime
def download_cci(lat_boundaries, lon_boundaries, time_init_date, time_final_date):
    """Downloads chl data from CCI v5 4km using previously defined
    Region of Interest and Time Period by user"""
    # Open netcdf4 file using OPENDAP
    nc_in = nc4.Dataset('https://www.oceancolour.org/thredds/dodsC/CCI_ALL-v5.0-DAILY')
    # Extract latitude and longitude
    lati = nc_in.variables['lat'][:]
    loni = nc_in.variables['lon'][:]
    lat_lb = np.argmin(abs(lati-lat_boundaries[0])) #sets latitude lower boundary
    lat_ub = np.argmin(abs(lati-lat_boundaries[1])) #sets latitude upper boundary
    lon_lb = np.argmin(abs(loni-lon_boundaries[0])) #sets longitude lower boundary
    lon_ub = np.argmin(abs(loni-lon_boundaries[1])) #sets longitude lower boundary
    lon = np.array(nc_in.variables['lon'][lon_lb:lon_ub])
    lat = np.array(nc_in.variables['lat'][lat_lb:lat_ub])
    # Extract time
    time_total = np.array(nc_in.variables['time'][:])
    time_total_date = np.empty(len(time_total), dtype=np.object)
    for i, item in enumerate(time_total):
        time_total_date[i] = serial_date_to_string(int(time_total[i]))
    time_start_ind = np.where(time_total_date == time_init_date)[0][0]
    time_start_end = np.where(time_total_date == time_final_date)[0][0]+1
    time_array = np.array(nc_in.variables['time'][time_start_ind:time_start_end])
    time_array_date = np.empty(len(time_array), dtype=np.object)
    for i, item in enumerate(time_array):
        time_array_date[i] = serial_date_to_string(int(time_array[i]))
    chl = np.array(nc_in.variables['chlor_a'][time_start_ind:time_start_end,
                                              lat_lb:lat_ub, lon_lb:lon_ub])
    # Swaps axes to lon, lat, time
    chl = np.swapaxes(np.swapaxes(chl, 0, 2), 0, 1)
    # Replaces invalid values with NaNs
    chl[chl == 9.96921E36] = np.nan
    return chl, lat, lon, time_array, time_array_date

## First step: define the boundaries of region of interest (ROI)

Provide the latitude/longitude coordinates for each corner of your area (has to be a square).

In [3]:
### Define ROI
#Please enter upper right corner latitude [-90-90°N]:
lat_max = '44'
#Please enter lower left corner latitude [-90-90°N]:
lat_min = '36'
#Please enter upper right corner longitude [-180-180°E]:
lon_max = '-12'
#Please enter lower left corner longitude [-180-180°E]:
lon_min = '-6'
LATBD, LONBD = define_ROI(lat_max, lat_min, lon_max, lon_min)

Your region of interest is:
Latitude: [44, 36]
Longitude: [-6, -12]


## Second step: define the temporal coverage of the data

Provide the first and last day of the data which you want.

Note: The dataset will include the last day.

In [4]:
### Define timespan
# Please enter initial day [YYYY-MM-DD]:
time_start = '2018-10-10'
# Please enter final day [YYYY-MM-DD]:
time_end = '2018-10-11'
time_start_datetime, time_end_datetime = define_time(time_start, time_end)

Your data timespan is:
Initial date: 2018-10-10
Final date: 2018-10-11
That corresponds to 2 days.


## Download the data

Important: do not try to download very large datasets as the OC-CCI will block them.

If you want to download a large dataset, break it down in smaller time periods and use the script to join the smaller datasets (*OC-CCI_download_join.ipynb*)

In [5]:
#Please enter the desired name for the downloaded file
filename_out_chl = 'cciv5_4km_chl_test'
chl, lat, lon, time_array, time_array_date = download_cci(LATBD,
                                                          LONBD,
                                                          time_start_datetime,
                                                          time_end_datetime)

C:\Users\afons\AppData\Local\Temp/ipykernel_10308/3581257380.py:48: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lati = nc_in.variables['lat'][:]
C:\Users\afons\AppData\Local\Temp/ipykernel_10308/3581257380.py:49: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  loni = nc_in.variables['lon'][:]
C:\Users\afons\AppData\Local\Temp/ipykernel_10308/358125

## Save the data

By default, the dataset will save to your downloads folder. You can change this in the code if you want.

In [6]:
### Save data in Downloads Folder by default
os.chdir(str(Path.home() / "Downloads"))
np.savez_compressed(filename_out_chl, lat=lat, lon=lon, chl=chl,
                    time=time_array, time_date=time_array_date)

Note that this script will save files in numpy compressed files, a python-specific format which allows for smaller files.

You can open them by using the following commands and then use the same commands that you learned in the previous Jupyter Notebooks.

```python
# Make sure the name of your file is correct
dataset = np.load('yourfile.npz',allow_pickle = True)
chl = dataset['chl']
lat = dataset['lat']
lon = dataset['lon']
time = dataset['time_date']
```